In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# Read product groups, train and test file
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

product_groups = pd.read_csv("../data/product_groups.csv")
product_groups = product_groups.astype(object)

product_groups

,category_number,category_level_1,category_level_2,category_level_3,category_level_4
0,9046,341,20,10,10
1,9009,125,25,10,10
2,9049,410,65,20,10
3,9019,300,10,19,11
4,9035,305,31,10,10
...,...,...,...,...,...
3908,9019,616,95,20,15
3909,9020,616,95,20,10
3910,9020,616,95,20,15
3911,9020,616,95,20,30


In [3]:
# Build mappings: 
# hash4: category_level1, level2, level3, level4 -> category_number 
# hash3: category_level1, level2, level3 -> category_number
# hash2: category_level1, level2 -> category_number
# hash1: category_level1 -> category_number

product_groups["hash1"] = product_groups["category_level_1"].astype(str)
product_groups["hash2"] = product_groups["category_level_1"].astype(str) + (product_groups["category_level_2"].astype(str))
product_groups["hash3"] = product_groups["category_level_1"].astype(str) + (product_groups["category_level_2"].astype(str)) + (product_groups["category_level_3"].astype(str))
product_groups["hash4"] = product_groups["category_level_1"].astype(str) + (product_groups["category_level_2"].astype(str)) + (product_groups["category_level_3"].astype(str)) + (product_groups["category_level_4"].astype(str))

hash1_to_category_number = {}
hash2_to_category_number = {}
hash3_to_category_number = {}
hash4_to_category_number = {}

for row in product_groups[["category_number", "hash1"]].itertuples():
    hash = row[2]
    if hash not in hash1_to_category_number:
        hash1_to_category_number[hash] = []

    if row[1] not in hash1_to_category_number[hash]:
        hash1_to_category_number[hash].append(row[1])

for row in product_groups[["category_number", "hash2"]].itertuples():
    hash = row[2]
    if hash not in hash2_to_category_number:
        hash2_to_category_number[hash] = []

    if row[1] not in hash2_to_category_number[hash]:
        hash2_to_category_number[hash].append(row[1])

for row in product_groups[["category_number", "hash3"]].itertuples():
    hash = row[2]
    if hash not in hash3_to_category_number:
        hash3_to_category_number[hash] = []

    if row[1] not in hash3_to_category_number[hash]:
        hash3_to_category_number[hash].append(row[1])

for row in product_groups[["category_number", "hash4"]].itertuples():
    hash = row[2]
    if hash not in hash4_to_category_number:
        hash4_to_category_number[hash] = []

    if row[1] not in hash4_to_category_number[hash]:
        hash4_to_category_number[hash].append(row[1])

In [4]:
# Get card number - individual number match:

shopping_by_card_number = pd.read_csv("../data/transaction_header.csv")
shopping_by_card_number = shopping_by_card_number.astype(object)
print(shopping_by_card_number.describe()) # Each basket id is unique 
print()

cardnumber_to_individualnumber = pd.read_csv("../data/customeraccount.csv")

cardnumber_to_individualnumber = cardnumber_to_individualnumber.astype(object)
print(cardnumber_to_individualnumber.describe()) # Card number is unique,
# But some individuals have more than one cards: 28593 individuals, 35159 cards.
cardnumber_to_individualnumber

       date_of_transaction        cardnumber           basketid  is_sanal
count              1124673           1124673            1124673   1124673
unique                 366             30319            1124673         2
top             2020-12-31  9887854616568815  20120163990041163         0
freq                  5032              1239                  1   1024395

        individualnumber        cardnumber
count              35159             35159
unique             28593             35159
top             96035559  7287134635560315
freq                  49                 1


,individualnumber,cardnumber
0,16481068,7287134635560315
1,17634947,6587004647560415
2,19323290,8187034648564315
3,19323290,3005502487247749
4,37820213,6287114610560316
...,...,...
35154,100998656,2687784602565315
35155,101074070,6387894671560235
35156,101270229,9287054697566815
35157,101328888,2587064608564915


In [5]:
# Get: 
# basket id - individual number match
# individual_number - sanal and non-sanal shopping counts match
# individual_number - days_since_last_shopping match
# individual_number - shopping_range match

# Basket id - individual number match:
shopping_by_individual_number = pd.merge(shopping_by_card_number,cardnumber_to_individualnumber,how='inner',on='cardnumber')
shopping_by_individual_number = shopping_by_individual_number[["individualnumber", "basketid", "is_sanal", "date_of_transaction"]]

basket_id_to_individual_number = shopping_by_individual_number[["basketid", "individualnumber"]].set_index("basketid")
basket_id_to_individual_number = basket_id_to_individual_number["individualnumber"].to_dict()
basket_id_to_individual_number

# Individual_number - sanal, non-sanal shopping counts:
individual_shopping_details = pd.DataFrame()

shopping_by_individual_number["is_sanal"] = shopping_by_individual_number["is_sanal"].astype(np.uint8)
by_individual = shopping_by_individual_number.groupby("individualnumber")
individual_sanal_mean = by_individual["is_sanal"].mean().to_dict()

individual_shopping_details["individualnumber"] = individual_sanal_mean.keys()
individual_shopping_details["sanal_percent"] = individual_sanal_mean.values()
individual_shopping_details["shop_count"] = by_individual["basketid"].count().values

# Individual_number - months_since_last_shopping (from 2022 january):

last_shop_dates = by_individual["date_of_transaction"].max()

individual_to_months_since_last_shopping = {}
for individual, last_date in last_shop_dates.items():
    last_date_str = str(last_date)
    if last_date_str == "2021-12-01":
        last_date_str = "2021-11-30" 
    
    year = int(last_date_str[:4])
    month = int(last_date_str[5:7])
    months_since_last_shopping = 12 * (2021 - year) + (12 - month)
    individual_to_months_since_last_shopping[individual] = months_since_last_shopping

individual_shopping_details["months_since_last_shopping"] = individual_to_months_since_last_shopping.values()

# Individual_number - shopping range:
first_shop_dates = by_individual["date_of_transaction"].min()
individual_to_shopping_range = {}

for individual, first_date in first_shop_dates.items():
    first_date_str = str(first_date)
    f_year = int(first_date_str[:4])
    f_month = int(first_date_str[5:7])
    f_day = int(first_date_str[8:])
    first_date = datetime.date(f_year, f_month, f_day)


    last_date_str = last_shop_dates[individual]
    l_year = int(last_date_str[:4])
    l_month = int(last_date_str[5:7])
    l_day = int(last_date_str[8:])
    last_date = datetime.date(l_year, l_month, l_day)

    delta_days = (last_date - first_date).days
    individual_to_shopping_range[individual] = delta_days + 1

individual_shopping_details["days_shopped"] = individual_to_shopping_range.values()

# Resulting df:
individual_shopping_details


,individualnumber,sanal_percent,shop_count,months_since_last_shopping,days_shopped
0,30690,0.000000,42,1,359
1,294867,0.160000,50,1,365
2,322878,0.138889,72,1,332
3,356411,0.265823,79,1,360
4,374368,0.166667,90,1,360
...,...,...,...,...,...
28571,102278919,0.000000,1,1,1
28572,102279267,1.000000,1,1,1
28573,102280492,0.000000,1,1,1
28574,102280832,0.000000,2,1,5


In [6]:
# Read transaction sale file:
transaction_sale = pd.read_csv("../data/transaction_sale/transaction_sale.csv")

# Some discount_type_1 entries are empty: fill them with 0
transaction_sale["discount_type_1"].fillna(0, inplace=True)
transaction_sale.head()

,basketid,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,20120706070177471,613,50,50,15,19.9,2.0,0.0,0.0,0.0
1,20120706070177471,425,30,12,10,3.5,1.0,0.0,0.0,0.0
2,20120706070177471,425,30,12,11,7.9,1.0,0.0,0.0,0.0
3,20122703940031323,103,25,15,10,7.9,2.0,0.0,0.0,0.0
4,20122703940031323,101,15,15,15,16.0,1.0,0.0,0.0,0.0


* Implemented three methods for storing the shopping history of a customer.
* Store Type 3 is used in current implementation

### Store Type 1: _Store each buy seperately_

In this format data is stored in the following way:

    {
        "individual_number_1": [
            {"category_1": 130, "category_2": 45, "category_3": 10, "category_4": 20, "amount": 19.90, "quantity": 1.000}
            {"category_1": 150, "category_2": 15, "category_3": 15, "category_4": 25, "amount": 3.50, "quantity": 1.430}
        ]
        "individual_number_2": [
            {"category_1": 617, "category_2": 25, "category_3": 15, "category_4": 10, "amount": 8.75, "quantity": 2.000}
        ]
    }

### Store Type 2: _Store by total volume of the buy, classify them according to category level 1 and category level 2_

In this format data is stored in the following way:

    {
        "individual_number_1": {

            613: {
                50: [3.0, 49.75], 
                40: [11.0, 248.65], 
                15: [2.0, 8.9]
            }, 

            425: {
                30: [2.0, 11.4]
            }, 
            
            130: {
                85: [10.0, 165.06]
            }
        }
    }

* First key represents the category level 1.
* Second key represents the category level 2.
* Inner list stores the quantity and total money spent on that combination of category_1, category_2:
* [quantity, total_amount]

### Store Type 3: _Store by total volume of the buy, classify them according to category number used in campaigns_

In this format data is stored in the following way:

    {
        "individual_number_1": {
            hash1:
                9049: [47.89, 3054.63],
                9013: [xx.xx, xxxx.xx],
                ...
                "unknown": [xxx.xx, xxxxx.xx]
            hash2:
                ...
            hash3:
                ...
            hash4:
                ...
        }
        
        "individual_number_2": {
            hash1:
                9049: [16.23, 2045.82],
                9013: [xx.xx, xxxx.xx],
                ...
                "unknown": [xxx.xx, xxxxx.xx]
            hash2:
                ...
            hash3:
                ...
            hash4:
                ...
        }        
    }

* Category level's are converted to category number used in campaigns by using 4 different hash methods desribed above.

In [7]:

# Store type 1: ~90s to execute.
# Store type 2: ~25s to execute.
# Store type 3: ~90s to execute.

# Select store type: 1 or 2 or 3:
store_type = 3

individual_shopping_information = {}
individual_shopping_volume = {} # Simple dictionary: {"individual_number": total_money_spent}
individual_to_total_discount = {} # Simple dictionary: {"individual_number": total_discount}

# Read the transaction_sale file and save each customer's shopping history in the "individual_shopping_information" dictionary.
for row in transaction_sale.itertuples(): 
    # Get necessary features:

    basket_id = row[1]
    category_1 = row[2]
    category_2 = row[3]
    category_3 = row[4]
    category_4 = row[5]
    amount = row[6]
    quantity = row[7]
    discount_1 = row[8]
    discount_2 = row[9]
    discount_3 = row[10]
    
    # Get total discount in terms of tl:
    total_discount_value = amount * quantity * (1 - (1 - (discount_1 / 100)) * (1 - (discount_2 / 100)) * (1 - (discount_3 / 100)))
    # Get individual number:
    individual_number = basket_id_to_individual_number[basket_id]
    
    if individual_number not in individual_shopping_volume:
        individual_shopping_volume[individual_number] = 0
    individual_shopping_volume[individual_number] += quantity * amount

    if store_type == 1:
        # Storing method: type1 -> Store the products seperately.
        # Initialize customer shopping information if customer is not in information_shopping_information:
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = []
            
        product_information = {}
        product_information["category_level_1"] = category_1
        product_information["category_level_2"] = category_2
        product_information["category_level_3"] = category_3
        product_information["category_level_4"] = category_4
        product_information["amount"] = amount
        product_information["quantity"] = quantity

        individual_shopping_information[individual_number].append(product_information)
    
    elif store_type == 2:
        # Storing method: type2 -> Store the products by keys as category_1 and category_2, values = volume of the buy history.
        # Initialize customer shopping information if customer is not in individual_shopping_information:        
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = {}
        
        # Initialize category_1 in customer's shopping history if category_1 is not in customer's shopping history:
        if category_1 not in individual_shopping_information[individual_number]:
            individual_shopping_information[individual_number][category_1] = {}

        # Initialize category_1 in customer's category_1 shopping history if category_2 is not in customer's category_1 shopping history:
        if category_2 not in individual_shopping_information[individual_number][category_1]:
             # 0th index will be quantity, 1st index will be total money spent on that category combination.
            individual_shopping_information[individual_number][category_1][category_2] = [0, 0]
        
        # Increment the total volume in customer's shopping history branch: category_1, branch: category_2:
        individual_shopping_information[individual_number][category_1][category_2][0] += quantity
        individual_shopping_information[individual_number][category_1][category_2][1] += quantity * amount

    elif store_type == 3:
        # Storing method: type3 -> Store the products' volume by category_number used in campaign.
        # To match a product's category levels to category type used in campaign, use different hash methods and save data using all 4 hashing methods.
        # Initialize customer shopping information if customer is not in individual_shopping_information:
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = {"hash1": {}, "hash2": {}, "hash3": {}, "hash4": {}}
            individual_to_total_discount[individual_number] = 0
        
        category_hash4 = str(category_1) + str(category_2) + str(category_3) + str(category_4)
        try:
            # If category_hash4 is found in hash4_to_category_number:
            category_number_list4 = hash4_to_category_number[category_hash4]
        except:
            # Else category_number is unknown
            category_number_list4 = ["unknown"]

        category_hash3 = str(category_1) + str(category_2) + str(category_3)
        try:
            # If category_hash3 is found in hash3_to_category_number:
            category_number_list3 = hash3_to_category_number[category_hash3]
        except:
            # Else category_number is unknown
            category_number_list3 = ["unknown"]    

        category_hash2 = str(category_1) + str(category_2)
        try:
            # If category_hash2 is found in hash2_to_category_number:
            category_number_list2 = hash2_to_category_number[category_hash2]
        except:
            # Else category_number is unknown
            category_number_list2 = ["unknown"]

        category_hash1 = str(category_1)
        try:
            # If category_hash1 is found in hash1_to_category_number:
            category_number_list1 = hash1_to_category_number[category_hash1]
        except:
            # Else category_number is unknown
            category_number_list1 = ["unknown"]

        # Initialize category_number in customer's shopping history if category_number is not in customer's shopping history:
        for category_number in category_number_list4:
            if category_number not in individual_shopping_information[individual_number]["hash4"]:
                # 0th index will be quantity, 1st index will be total money spent on that category number.
                individual_shopping_information[individual_number]["hash4"][category_number] = [0, 0]
            
            individual_shopping_information[individual_number]["hash4"][category_number][0] += quantity
            individual_shopping_information[individual_number]["hash4"][category_number][1] += quantity * amount

        # Initialize category_number in customer's shopping history if category_number is not in customer's shopping history:
        for category_number in category_number_list3:
            if category_number not in individual_shopping_information[individual_number]["hash3"]:
                # 0th index will be quantity, 1st index will be total money spent on that category number.
                individual_shopping_information[individual_number]["hash3"][category_number] = [0, 0]
            
            individual_shopping_information[individual_number]["hash3"][category_number][0] += quantity
            individual_shopping_information[individual_number]["hash3"][category_number][1] += quantity * amount

        # Initialize category_number in customer's shopping history if category_number is not in customer's shopping history:
        for category_number in category_number_list2:
            if category_number not in individual_shopping_information[individual_number]["hash2"]:
                # 0th index will be quantity, 1st index will be total money spent on that category number.
                individual_shopping_information[individual_number]["hash2"][category_number] = [0, 0]
            
            individual_shopping_information[individual_number]["hash2"][category_number][0] += quantity
            individual_shopping_information[individual_number]["hash2"][category_number][1] += quantity * amount

        # Initialize category_number in customer's shopping history if category_number is not in customer's shopping history:
        for category_number in category_number_list1:
            if category_number not in individual_shopping_information[individual_number]["hash1"]:
                # 0th index will be quantity, 1st index will be total money spent on that category number.
                individual_shopping_information[individual_number]["hash1"][category_number] = [0, 0]
            
            individual_shopping_information[individual_number]["hash1"][category_number][0] += quantity
            individual_shopping_information[individual_number]["hash1"][category_number][1] += quantity * amount            

        # Add total discount earned by customer:
        individual_to_total_discount[individual_number] += total_discount_value

In [8]:
# Prepare individual shopping volume and individual total discount data:
individual_shopping_volume_df = pd.DataFrame()
individual_shopping_volume_df["individualnumber"] = individual_shopping_volume.keys()
individual_shopping_volume_df["total_money_spent"] = individual_shopping_volume.values()

individual_total_discount_df = pd.DataFrame()
individual_total_discount_df["individualnumber"] = individual_to_total_discount.keys()
individual_total_discount_df["total_discount"] = individual_to_total_discount.values()

individual_total_discount_df.head(10)

,individualnumber,total_discount
0,90065509,3274.793136
1,56076820,368.561556
2,37752547,749.005609
3,98384331,156.819866
4,94255308,110.135029
5,93937111,3401.739191
6,94688545,97.065689
7,91747483,3073.118014
8,94778614,5237.759674
9,16893075,178.903728


In [9]:
# Collect and organize individual personal information data:

# Convert birth date into age:
customer_personal = pd.read_csv("../data/customer.csv")
customer_personal["age"] = 2022 - customer_personal["dateofbirth"]
customer_personal.drop(columns="dateofbirth", inplace=True)

# Replace city missing values with 34 (Istanbul) (This method may change)
customer_personal["city_code"].fillna(34, inplace=True)

customer_personal.head()

,individualnumber,gender,city_code,age
0,94212124,K,34.0,76.0
1,96387515,E,34.0,48.0
2,95040383,E,80.0,42.0
3,94694434,E,34.0,52.0
4,47648671,E,35.0,59.0


In [15]:
# Organizes the test or train data with the features prepared.
# parameter df -> is the test data or train data.

def prepare_data(df):
    # To initialize the data, merge df (it is train or test) data with individual shopping volume (total money spent):
    data_organized = pd.merge(df, individual_shopping_volume_df, how="left", on="individualnumber")
    # Merge the data with individual total discount:
    data_organized = pd.merge(data_organized, individual_total_discount_df, how="left", on="individualnumber")
    # Merge the data with individual shopping details (sanal percentage, months since last shopping etc.)
    data_organized = pd.merge(data_organized, individual_shopping_details, how="left", on="individualnumber")
    # Merge the data with customer personal information (age, gender etc.): 
    data_organized = pd.merge(data_organized, customer_personal, how="left", on="individualnumber")

    # Prepeare relevant category, relevant quantity features:
    # relevant_category = The category number of the campaing that has been proposed to individual.

    # Includes total money spent by a customer in relevant_category. Calculate relevancy using combinations:
    # Level1, Level1 and Level2, Level1 and Level2 and Level3, Level1 and Level2 and Level3 and Level4:
    level1_relevant_category_volume_column = []
    level2_relevant_category_volume_column = []
    level3_relevant_category_volume_column = []
    level4_relevant_category_volume_column = []
    # Includes total quantity of items in relevant_category that is bought by customer. Calculate relevancy using combinations:
    # Level1, Level1 and Level2, Level1 and Level2 and Level3, Level1 and Level2 and Level3 and Level4:
    level1_relevant_category_quantity_column = []
    level2_relevant_category_quantity_column = []
    level3_relevant_category_quantity_column = []
    level4_relevant_category_quantity_column = []

    # Iterate over individuals to add more features into the data frame.
    for row in data_organized.itertuples():
        individual_number = row[1]
        category_number = row[2]
        total_money_spent = row[6]

        # Get relevant category expense from the data
        try:
            level1_relevant_money_spent = individual_shopping_information[individual_number]["hash1"][category_number][1]
            level1_relevant_quantity = individual_shopping_information[individual_number]["hash1"][category_number][0]
        except:
            level1_relevant_money_spent = 0
            level1_relevant_quantity = 0
        
        try:
            level2_relevant_money_spent = individual_shopping_information[individual_number]["hash2"][category_number][1]
            level2_relevant_quantity = individual_shopping_information[individual_number]["hash2"][category_number][0]
        except:
            level2_relevant_money_spent = 0
            level2_relevant_quantity = 0

        try:
            level3_relevant_money_spent = individual_shopping_information[individual_number]["hash3"][category_number][1]
            level3_relevant_quantity = individual_shopping_information[individual_number]["hash3"][category_number][0]
        except:
            level3_relevant_money_spent = 0
            level3_relevant_quantity = 0

        try:
            level4_relevant_money_spent = individual_shopping_information[individual_number]["hash4"][category_number][1]
            level4_relevant_quantity = individual_shopping_information[individual_number]["hash4"][category_number][0]
        except:
            level4_relevant_money_spent = 0
            level4_relevant_quantity = 0

        # Append columns by relevant information:
        level1_relevant_category_volume_column.append(round(level1_relevant_money_spent, 3))
        level1_relevant_category_quantity_column.append(round(level1_relevant_quantity, 3))

        level2_relevant_category_volume_column.append(round(level2_relevant_money_spent, 3))
        level2_relevant_category_quantity_column.append(round(level2_relevant_quantity, 3))

        level3_relevant_category_volume_column.append(round(level3_relevant_money_spent, 3))
        level3_relevant_category_quantity_column.append(round(level3_relevant_quantity, 3))

        level4_relevant_category_volume_column.append(round(level4_relevant_money_spent, 3))
        level4_relevant_category_quantity_column.append(round(level4_relevant_quantity, 3))
            
    # Finalize columns in the original data frame:
    data_organized["level1_relevant_category_volume"] = level1_relevant_category_volume_column
    data_organized["level2_relevant_category_volume"] = level2_relevant_category_volume_column
    data_organized["level3_relevant_category_volume"] = level3_relevant_category_volume_column
    data_organized["level4_relevant_category_volume"] = level4_relevant_category_volume_column

    data_organized["level1_relevant_category_quantity"] = level1_relevant_category_quantity_column
    data_organized["level2_relevant_category_quantity"] = level2_relevant_category_quantity_column
    data_organized["level3_relevant_category_quantity"] = level3_relevant_category_quantity_column
    data_organized["level4_relevant_category_quantity"] = level4_relevant_category_quantity_column

    # Create new features using the existing ones.
    data_organized["level1_relevant_category_volume_per_day"] = data_organized["level1_relevant_category_volume"] / data_organized["days_shopped"]
    data_organized["level2_relevant_category_volume_per_day"] = data_organized["level2_relevant_category_volume"] / data_organized["days_shopped"]
    data_organized["level3_relevant_category_volume_per_day"] = data_organized["level3_relevant_category_volume"] / data_organized["days_shopped"]
    data_organized["level4_relevant_category_volume_per_day"] = data_organized["level4_relevant_category_volume"] / data_organized["days_shopped"]

    data_organized["level1_relevant_category_quantity_per_day"] = data_organized["level1_relevant_category_quantity"] / data_organized["days_shopped"]
    data_organized["level2_relevant_category_quantity_per_day"] = data_organized["level2_relevant_category_quantity"] / data_organized["days_shopped"]
    data_organized["level3_relevant_category_quantity_per_day"] = data_organized["level3_relevant_category_quantity"] / data_organized["days_shopped"]
    data_organized["level4_relevant_category_quantity_per_day"] = data_organized["level4_relevant_category_quantity"] / data_organized["days_shopped"]

    data_organized["total_money_spent_per_day"] = data_organized["total_money_spent"] / data_organized["days_shopped"]
    data_organized["discount_per_day"] = data_organized["total_discount"] / data_organized["days_shopped"]

    data_organized["odul/hakkedis"] = data_organized["odul_amt"] / data_organized["hakkedis_amt"]

    data_organized["is_large_city"] = ((data_organized["city_code"] == 34) | (data_organized["city_code"] == 35) | (data_organized["city_code"] == 6) | (data_organized["city_code"] == 7) | (data_organized["city_code"] == 16))

    return data_organized

In [19]:
# Prepare the train data:
train_organized = prepare_data(train)
train_organized

,individualnumber,category_number,hakkedis_amt,odul_amt,response,total_money_spent,total_discount,sanal_percent,shop_count,months_since_last_shopping,...,level3_relevant_category_volume_per_day,level4_relevant_category_volume_per_day,level1_relevant_category_quantity_per_day,level2_relevant_category_quantity_per_day,level3_relevant_category_quantity_per_day,level4_relevant_category_quantity_per_day,total_money_spent_per_day,discount_per_day,odul/hakkedis,is_large_city
0,94230288,9000,21.0,2.0,0,7118.94741,177.489978,0.015625,64.0,1.0,...,0.675378,0.675378,0.270924,0.131653,0.100840,0.100840,19.941029,0.497171,0.095238,True
1,4684087,9000,17.0,1.0,0,10192.64563,31907.260487,0.000000,30.0,1.0,...,0.342718,0.342718,0.113528,0.038835,0.038835,0.038835,32.985908,103.259743,0.058824,False
2,92472145,9058,24.0,3.0,0,35113.80415,10060.118391,0.013514,518.0,1.0,...,1.806940,0.330191,3.309934,0.396175,0.368852,0.030055,95.939356,27.486662,0.125000,True
3,88026681,9030,22.0,2.0,0,11559.66676,231.976316,0.950000,40.0,1.0,...,0.396143,0.396143,0.040000,0.022857,0.022857,0.022857,33.027619,0.662789,0.090909,True
4,98127795,9001,38.0,3.0,0,461.77380,1.738800,0.000000,4.0,3.0,...,0.202128,0.202128,0.058511,0.010638,0.010638,0.010638,2.456244,0.009249,0.078947,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13110,97214433,9059,27.0,2.0,0,16893.83688,1598.015176,0.009881,506.0,1.0,...,0.232356,0.161534,0.060274,0.019178,0.019178,0.013699,46.284485,4.378124,0.074074,True
13111,96841665,9004,65.0,6.0,0,13089.84113,500.344604,0.000000,40.0,1.0,...,0.748507,0.748507,0.065672,0.014925,0.014925,0.014925,39.074153,1.493566,0.092308,True
13112,98445787,9044,33.0,3.0,0,2432.60788,244.758629,0.000000,39.0,1.0,...,0.077468,0.077468,0.030342,0.010734,0.002331,0.002331,6.814028,0.685598,0.090909,False
13113,13781030,9004,66.0,6.0,0,1242.70635,6.080452,0.090909,11.0,1.0,...,0.202663,0.202663,0.002959,0.002959,0.002959,0.002959,3.676646,0.017990,0.090909,False


In [17]:
# Prepare the test data:
test_organized = prepare_data(test)
test_organized

,individualnumber,category_number,hakkedis_amt,odul_amt,total_money_spent,total_discount,sanal_percent,shop_count,months_since_last_shopping,days_shopped,...,level3_relevant_category_volume_per_day,level4_relevant_category_volume_per_day,level1_relevant_category_quantity_per_day,level2_relevant_category_quantity_per_day,level3_relevant_category_quantity_per_day,level4_relevant_category_quantity_per_day,total_money_spent_per_day,discount_per_day,odul/hakkedis,is_large_city
0,95812936,9029,38.0,3.0,2408.92000,37.395403,0.000000,4.0,4.0,77.0,...,0.000000,0.000000,0.025974,0.012987,0.000000,0.000000,31.284675,0.485655,0.078947,False
1,23929570,9012,28.0,2.0,1317.56637,175.519563,0.000000,21.0,1.0,338.0,...,0.000000,0.000000,0.035503,0.000000,0.000000,0.000000,3.898125,0.519289,0.071429,True
2,95948115,9019,21.0,2.0,553.84536,28.904031,0.000000,9.0,8.0,98.0,...,0.000000,0.000000,0.074898,0.023622,0.000000,0.000000,5.651483,0.294939,0.095238,False
3,100898513,9035,48.0,4.0,18436.09635,1363.276980,0.000000,41.0,1.0,232.0,...,3.396991,3.396991,0.262414,0.262414,0.253793,0.253793,79.465933,5.876194,0.083333,False
4,12769156,9015,26.0,3.0,118087.50388,6966.507346,0.088462,260.0,1.0,365.0,...,0.247397,0.247397,1.317271,0.052055,0.019178,0.019178,323.527408,19.086321,0.115385,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13231,93418529,9049,32.0,3.0,4007.74538,207.789769,0.000000,56.0,1.0,344.0,...,0.333663,0.333663,0.031977,0.031977,0.031977,0.031977,11.650423,0.604040,0.093750,False
13232,97397069,9018,20.0,2.0,239.44000,0.420064,0.000000,3.0,2.0,299.0,...,0.068729,0.068729,0.010033,0.010033,0.010033,0.010033,0.800803,0.001405,0.100000,False
13233,95212479,9046,85.0,8.0,330.94000,30.524957,0.000000,5.0,2.0,315.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.050603,0.096905,0.094118,True
13234,93406449,9032,39.0,3.0,1470.80640,67.445450,0.000000,25.0,1.0,317.0,...,0.658328,0.658328,0.041009,0.022082,0.018927,0.018927,4.639768,0.212762,0.076923,False


In [20]:
# Output the prepared data into csv files:
path = "../organized_data"

train_organized.to_csv(f"{path}/train_organized.csv", index=False)
test_organized.to_csv(f"{path}/test_organized.csv", index=False)